In [1]:
import requests  # library to handle requests
import pandas as pd # library for data analsysis

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]
df.to_csv('postalcodescanada.csv')    #Import data and store the resulting data in data frame
df1=df
df1['Borough'] = df1['Borough'].replace({'Not assigned':None}) #Replacing 'Not assigned' value in 'Borough' column with null value
df1.dropna(axis=0, inplace=True)
df1 = df1.reset_index()     # Re index the data frame after dropping null values from Borough column
del df1['index']
df1['Neighbourhood'] = df1['Neighbourhood'].replace({'Not assigned':None}) #Replace Not assigned values in Neighbourhood column to null
df1.Neighbourhood.fillna(df1.Borough, inplace=True)   # Replace null value with corresponding column values for Neighbourhood
df1 = df1.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index() #To group postal code data
#Import coordinates data into data frame
df_location=pd.read_csv('http://cocl.us/Geospatial_data')
#Join two data frames
df2=pd.merge(df1, df_location, left_on='Postcode', right_on='Postal Code', how='left').drop(columns='Postal Code', axis=1)
df2.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!conda install -c conda-forge geopy --yes
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('All Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

All Libraries imported.


In [3]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [4]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [5]:
# To map data around Central Toronto
toronto_data = df2[df2['Borough'] == 'Central Toronto'].reset_index(drop=True)
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
5,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936
7,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


In [6]:
# PLot the data on the map of toronto
map_centraltoronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_centraltoronto)  
    
map_centraltoronto

In [7]:
# Get a location in central Toronto region
toronto_data.loc[0, 'Neighbourhood']

'Lawrence Park'

In [12]:
# Get coordinates of the location
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


In [14]:
CLIENT_ID = 'A0NMPZK3CUIRSM0U5IOLL3O3T5SVZDEZ2MTFVW1JKMEZ31I0' # your Foursquare ID
CLIENT_SECRET = 'MREDF5DG4VRZA2LT1BWI1CJOGBTFZ2DFO4PCRF1GDS1EHS4K' # your Foursquare Secret
VERSION = '20190919' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A0NMPZK3CUIRSM0U5IOLL3O3T5SVZDEZ2MTFVW1JKMEZ31I0
CLIENT_SECRET:MREDF5DG4VRZA2LT1BWI1CJOGBTFZ2DFO4PCRF1GDS1EHS4K


In [15]:
# Url to get top 10 venues around Lawrence park with in a radius of 500mtrs
radius=500
LIMIT = 10
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=A0NMPZK3CUIRSM0U5IOLL3O3T5SVZDEZ2MTFVW1JKMEZ31I0&client_secret=MREDF5DG4VRZA2LT1BWI1CJOGBTFZ2DFO4PCRF1GDS1EHS4K&v=20190917&ll=43.73,-79.39&radius=500&limit=10'.format(CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=A0NMPZK3CUIRSM0U5IOLL3O3T5SVZDEZ2MTFVW1JKMEZ31I0&client_secret=MREDF5DG4VRZA2LT1BWI1CJOGBTFZ2DFO4PCRF1GDS1EHS4K&v=20190917&ll=43.73,-79.39&radius=500&limit=10'

In [16]:
import json # library to handle JSON files
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d8338ded0a14900394d2f91'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.7345000045, 'lng': -79.38378415510307},
   'sw': {'lat': 43.725499995499995, 'lng': -79.39621584489693}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'l

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Ecig Canada Zone,Electronics Store,43.731561,-79.395360


Conclusion : After normalizing 2 top venues have been identified around Central Toronto region

One is a Public Park and another one is an Electric Store